In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cd /content

/content


In [ ]:
!kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              215KB  2021-03-10 12:43:30            193  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            193  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            313  
nickuzmenkov/nih-chest-xrays-tfrecords                      NIH Chest X-rays TFRecords                         11GB  2021-03-09 04:49:23             41  
nickuzmenkov/ranzcr-clip-kfold-tfrecords                    RANZCR CLiP KFol

In [ ]:
!kaggle competitions download -c fcis-cs-deeplearningcompetition

 99% 558M/563M [00:05<00:00, 113MB/s] 
100% 563M/563M [00:05<00:00, 116MB/s]
  0% 0.00/8.26k [00:00<?, ?B/s]
100% 8.26k/8.26k [00:00<00:00, 8.90MB/s]
100% 157M/157M [00:01<00:00, 117MB/s]
100% 157M/157M [00:01<00:00, 129MB/s]
  0% 0.00/10.0 [00:00<?, ?B/s]
100% 10.0/10.0 [00:00<00:00, 8.33kB/s]


In [ ]:
!mkdir train
!mkdir test
!unrar x /content/train.rar -d /content/train/
!unrar x /content/test.rar -d /content/test/


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/train.rar

Creating    /content/train/train                                      OK
Creating    /content/train/train/airport_inside                       OK
Extracting  /content/train/train/airport_inside/airport_inside_0002.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0003.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0004.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0005.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0006.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0007.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0008.jpg       0%  OK 
Extracting  /content/train/train/airport_inside/airport_inside_0010.jpg       0%

In [ ]:
import numpy as np 
import pandas as pd 
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import applications
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop, SGD
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
from keras.layers import Input
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing import image
from tensorflow.python.keras import regularizers

IC=[]
for d in os.listdir('/content/train/train'):
  c=0
  P=os.path.join('/content/train/train',d)
  for i in os.listdir(P):
    #print(i)
    c+=1
  IC.append([d,c])
print(IC)

[['greenhouse', 405], ['toystore', 554], ['restaurant', 411], ['bedroom', 530], ['airport_inside', 487], ['poolinside', 411], ['kitchen', 557], ['bakery', 405], ['operating_room', 398], ['gym', 463]]


In [ ]:
Num=0
for i in IC:
  Num+=i[1]
print(Num)

3138


In [ ]:
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from skimage.io import imread, imsave
import cv2
import imutils
from scipy import ndimage
from scipy import misc
import os
import imgaug.augmenters as iaa

def Aug1(class_name):

  c=0
  for IM in os.listdir('/content/train/train/'+class_name):
    img = cv2.imread("/content/train/train/"+class_name+'/'+IM)
    if img is None:
      print('none')
      os.remove("/content/train/train/"+class_name+'/'+IM)
    else:
      if c%2==0: #using different 2 aug. based on index
        # Use Flip code 1 to flip horizontally 
        img = cv2.flip(img, 1) 
        alpha = 1 # Contrast control (1.0-3.0)
        beta = 50 # Brightness control (0-100)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        imsave("/content/train/train/"+class_name+'/'+IM+'Aug1.jpeg', img)
      else:
        img = imutils.rotate(img, 20)
        alpha = 1 # Contrast control (1.0-3.0)
        beta = 30 # Brightness control (0-100)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        imsave("/content/train/train/"+class_name+'/'+IM+'Aug2.jpeg', img)
      print(IM)
      print(c)
    c+=1

In [ ]:
for IM in os.listdir('/content/train/train/operating_room/'):
  if 'Aug2' in IM or 'Aug1' in IM or 'Aug3' in IM or 'Aug4' in IM:
    os.remove("/content/train/train/operating_room/"+IM)

In [ ]:
c=0
for IM in os.listdir('/content/train/train/kitchen/'):
  if c<=30:
    os.remove("/content/train/train/kitchen/"+IM)
    c+=1

In [ ]:
def Aug2(class_name):
  c=0
  for IM in os.listdir("/content/train/train/"+class_name):
    img = cv2.imread("/content/train/train/"+class_name+'/'+IM)
    if img is None:
      print('none')
      os.remove("/content/train/train/"+class_name+'/'+IM)
    else:
      if c%2==0: #using different 2 aug. based on index
        contrast=iaa.GammaContrast(gamma=2.0)
        contrast_image =contrast.augment_image(img)
        imsave("/content/train/train/"+class_name+'/'+IM+'Aug3.jpeg', contrast_image)
        #cv2_imshow(contrast_image)
      elif c%3==0:
        alpha = 1 # Contrast control (1.0-3.0)
        beta = 30 # Brightness control (0-100)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        imsave("/content/train/train/"+class_name+'/'+IM+'Aug5.jpeg', img)
      else:
        scale_im=iaa.Affine(scale={"x": (1.5, 1.0), "y": (1.5, 1.0)})
        img =scale_im.augment_image(img)
        imsave("/content/train/train/"+class_name+'/'+IM+'Aug4.jpeg', img)
        print(IM)
        print(c)
    c+=1

In [ ]:
def Aug3(class_name):
  c=0
  for IM in os.listdir("/content/train/train/"+class_name):
    img = cv2.imread("/content/train/train/"+class_name+'/'+IM)
    if img is None:
      print('none')
      os.remove("/content/train/train/"+class_name+'/'+IM)
    else:
      if c%4==0: #using different 2 aug. based on index
        alpha = 1.5 # Contrast control (1.0-3.0)
        beta = 30 # Brightness control (0-100)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        scale_im=iaa.Affine(scale={"x": (1.5, 1.0), "y": (1.5, 1.0)})
        img =scale_im.augment_image(img)
        imsave("/content/train/train/"+class_name+'/'+IM+'Aug6.jpeg', img)
    print(IM)
    print(c)
    c+=1


In [ ]:
Aug3('gym')

gimnasio_41_07_altavista.jpgAug1.jpeg
0
gimnasio_02_12_altavista.jpgAug1.jpeg
1
gym001.jpg
2
GymInt1.jpgAug2.jpeg
3
KoocanusaGym.jpgAug2.jpeg
4
gimnasio_35_23_flickr.jpg
5
gym_holiday_cottages.jpgAug1.jpeg
6
GYM_C.jpgAug1.jpeg
7
gimnasio_158_18_flickr.jpg
8
gimnasio_133_21_flickr.jpg
9
gimnasio_93_23_flickr.jpg
10
gimnasio_60_11_flickr.jpg
11
gimnasio_20_15_altavista.jpgAug2.jpeg
12
p1a.jpg
13
gimnasio_35_11_altavista.jpgAug2.jpeg
14
p1a.jpgAug1.jpeg
15
Gym_Equipment.jpg
16
gimnasio_134_13_flickr.jpgAug2.jpeg
17
gym04.jpgAug1.jpeg
18
Gym1_png.jpg
19
csu6.jpgAug2.jpeg
20
1852260398_75826537cd.jpg
21
gimnasio_07_17_altavista.jpg
22
Exercise_Gym.jpg
23
gym09.jpgAug2.jpeg
24
gym03.jpgAug2.jpeg
25
983000854_2bfd6a9ccf.jpgAug1.jpeg
26
DSC00139Gym1.jpgAug2.jpeg
27
gimnasio_98_14_flickr.jpgAug1.jpeg
28
room398.jpg
29
gimnasio_10_20_altavista.jpgAug1.jpeg
30
gimnasio_25_10_altavista.jpgAug1.jpeg
31
KoocanusaGym.jpg
32
gimnasio_19_09_altavista.jpgAug1.jpeg
33
gimnasio_46_02_altavista.jpgAug1.jpe

In [ ]:
import random

Train=[]
c=0
for N in os.listdir('/content/train/train'):
  L=np.zeros(10)
  L[c]=1
  print(L,N)
  c+=1
  for im in os.listdir("/content/train/train/"+N):
    Train.append(["/content/train/train/"+N+'/'+im,L])
print(len(Train))
random.shuffle(Train)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] greenhouse
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] toystore
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] restaurant
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] bedroom
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] airport_inside
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] poolinside
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] kitchen
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] bakery
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] operating_room
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] gym
4621


In [ ]:
from sklearn.model_selection import train_test_split
X=[]
Y=[]
for x,y in Train:
  X.append(x)
  Y.append(y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [81]:
from keras.utils import to_categorical

def Tgenerator(data,labels,batch_size):
    #c=0
    while True:
      for start in range(0, len(data),batch_size):
          count = 0
          batch_data = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
          batch_labels = np.zeros((batch_size,10), dtype=np.float32)
          end = min(start + batch_size, len(data))
          for i in range(start, end):
            img_name = data[i]
            label = labels[i]
            #print(label)
            #print(label,img_name)
            # one hot encoding
            #encoded_label = to_categorical(label, num_classes=10)
            # read the image and resize
            img = cv2.imread(img_name)
            img = cv2.resize(img, (224,224))
            
            # check if it's grayscale
            if img.shape[2]==1:
                img = np.dstack([img, img, img]) #convert to 3 channels.
            
            # cv2 reads in BGR mode by default
            orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # normalize the image pixels
            orig_img = img.astype(np.float32)/255.
            
            batch_data[count] = orig_img
            batch_labels[count] = label
            count+=1
          yield  batch_data, batch_labels




In [107]:
from keras.utils import to_categorical

def Vgenerator(data,labels):
  X_Val=[]
  Y_Val=[]
  for i in range(len(data)):
    img_name = data[i]
    label = labels[i]

    img = cv2.imread(img_name)
    img = cv2.resize(img, (224,224))
    
    # check if it's grayscale
    if img.shape[2]==1:
        img = np.dstack([img, img, img]) #convert to 3 channels.
    
    # cv2 reads in BGR mode by default
    orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # normalize the image pixels
    orig_img = img.astype(np.float32)/255.
    
    X_Val.append(orig_img)
    Y_Val.append(label)
  return np.array(X_Val),np.array(Y_Val)
        

In [ ]:
d,l=Tgenerator(X_test,y_test,16)
print(d[0],l[0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[[[0.06666667 0.10980392 0.00784314]
  [0.06666667 0.10980392 0.00784314]
  [0.07450981 0.10980392 0.00392157]
  ...
  [0.3882353  0.627451   0.7019608 ]
  [0.39215687 0.627451   0.69411767]
  [0.39215687 0.627451   0.6901961 ]]

 [[0.06666667 0.10980392 0.00784314]
  [0.07058824 0.10980392 0.00784314]
  [0.07450981 0.10980392 0.00392157]
  ...
  [0.3882353  0.6313726  0.7019608 ]
  [0.39607844 0.6313726  0.69411767]
  [0.39215687 0.627451   0.6901961 ]]

 [[0.07450981 0.11372549 0.0

In [ ]:
from keras import backend as K
#define another metrices
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [69]:
import glob
import h5py
import shutil
import imgaug as aug
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mimg
import imgaug.augmenters as iaa
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from keras.applications import VGG19
from keras import applications
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop, SGD
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
from keras.layers import Input
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing import image
from tensorflow.python.keras import regularizers
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping


In [115]:

input_tensor = Input(shape=(224, 224, 3))
vgg_conv = VGG19(weights='imagenet',
                               include_top=False,
                               input_tensor=input_tensor)
# Freeze the layers except the last 2 layers
for layer in vgg_conv.layers[:-2]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

# Create the model
model = models.Sequential()
# Add the vgg convolutional base model
model.add(vgg_conv)
model.add(layers.Dropout(0.5))
model.add(SeparableConv2D(128, (3,3), activation='relu', padding='same'))
model.add(SeparableConv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(SeparableConv2D(256, (3,3), activation='relu', padding='same'))
#model.add(BatchNormalization())
#model.add(SeparableConv2D(256, (3,3), activation='relu', padding='same'))
#model.add(BatchNormalization())

#model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same')) 
#model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same')) #best without that 85
model.add(MaxPooling2D((2,2)))



model.add(Flatten())
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.7))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(10, activation='softmax'))


<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fa3bd363810> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd363d10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd447410> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fa3bd3c3510> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd3c37d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd37a890> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fa3bd37a550> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd371e10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd36ee50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd3107d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fa3bd3841d0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 

In [71]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
dropout_8 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
separable_conv2d_14 (Separab (None, 7, 7, 128)         70272     
_________________________________________________________________
separable_conv2d_15 (Separab (None, 7, 7, 128)         17664     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
separable_conv2d_16 (Separab (None, 3, 3, 256)         34176     
_________________________________________________________________
batch_normalization_5 (Batch (None, 3, 3, 256)        

In [116]:
opt = optimizers.Adam(lr=0.0001, decay=1e-5)
#es = EarlyStopping(patience=5)
#chkpt = ModelCheckpoint(filepath='indoorScenes', save_best_only=True, save_weights_only=True)
model.compile(loss='categorical_crossentropy', metrics=['acc'],optimizer=opt) 

In [110]:
batch_size = 32

# Get a train data generator
train_data_gen = Tgenerator(X_train, y_train,batch_size)
X_val,Y_val = Vgenerator(X_test, y_test)

# Define the number of training steps
steps = len(X_train)//batch_size
print(steps)

115


In [111]:
print(X_val.shape)
print(len(Y_val))

(925, 224, 224, 3)
925


In [64]:
import multiprocessing as mp
mp.set_start_method('spawn', force=True)

In [120]:
import cv2
# Fit the model
history = model.fit_generator(train_data_gen, epochs=5, steps_per_epoch=steps,
                               validation_data=(X_val,Y_val)) #save the best one

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
115/115 [==============================] - 33s 288ms/step - loss: 0.4929 - acc: 0.8293 - val_loss: 0.5871 - val_acc: 0.7957
Epoch 2/5
115/115 [==============================] - 32s 283ms/step - loss: 0.4508 - acc: 0.8492 - val_loss: 0.5994 - val_acc: 0.8065
Epoch 3/5
115/115 [==============================] - 33s 287ms/step - loss: 0.4143 - acc: 0.8576 - val_loss: 0.5916 - val_acc: 0.8184
Epoch 4/5
115/115 [==============================] - 33s 288ms/step - loss: 0.3756 - acc: 0.8712 - val_loss: 0.5415 - val_acc: 0.8314
Epoch 5/5
115/115 [==============================] - 33s 286ms/step - loss: 0.3674 - acc: 0.8769 - val_loss: 0.5130 - val_acc: 0.8346


In [66]:
!pip uninstall keras

Uninstalling Keras-2.4.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.4.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.4.3


In [119]:
model.save('content/indoorScenes/81.h5')
model.save_weights('content/indoorScenes/81.h5')

In [183]:
import random

Test=[]
c=0
for im in os.listdir("/content/test/test/"):
    if not 'jpg' in im:
      print(im)
    Test.append(im)
print(len(Test))

772


In [182]:
os.rmdir('/content/test/test/.ipynb_checkpoints')

In [155]:
print(Test[0])

/content/test/test/restaurant_47_18_altavista.jpg


In [169]:
testing_data = []
for i in range(len(Test)):
    img_name = Test[i]
    print(img_name)
    img = cv2.imread(img_name)
    #print(img)
    img = cv2.resize(img, (224,224))
    
    # check if it's grayscale
    if img.shape[2]==1:
        img = np.dstack([img, img, img]) #convert to 3 channels.
    
    # cv2 reads in BGR mode by default
    orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # normalize the image pixels
    orig_img = img.astype(np.float32)/255.
    testing_data.append(orig_img)
testing_data=np.array(testing_data)

/content/test/test/restaurant_47_18_altavista.jpg
/content/test/test/cuarto_i_3.jpg
/content/test/test/boulangerie_04_12_altavista.jpg
/content/test/test/restaurant_11_19_altavista.jpg
/content/test/test/int273.jpg
/content/test/test/kitchen070.jpg
/content/test/test/restaurant_38_18_altavista.jpg
/content/test/test/panaderia_35_06_yahoo.jpg
/content/test/test/airport_inside_0009.jpg
/content/test/test/at_00_05_8293_15_l.jpg
/content/test/test/inside_pool.jpg
/content/test/test/room361.jpg
/content/test/test/OriginalWingRoom15_large.jpg
/content/test/test/airport_inside_0450.jpg
/content/test/test/bakery_02_03_altavista.jpg
/content/test/test/restaurante_02_14_altavista.jpg
/content/test/test/gym_b.jpg
/content/test/test/P1020153.jpg
/content/test/test/gimnasio_98_10_flickr.jpg
/content/test/test/gym_01_03_altavista.jpg
/content/test/test/gimnasio_18_07_altavista.jpg
/content/test/test/IMG_4830.jpg
/content/test/test/int492.jpg
/content/test/test/restaurant_44_20_altavista.jpg
/content

In [205]:
preds = model.predict(testing_data, batch_size=16)
preds = np.argmax(preds, axis=-1)

# Original labels
#orig_test_labels = np.argmax(test_labels, axis=-1)

#print(orig_test_labels.shape)
print(preds.shape)
print(preds)

(772,)
[2 5 4 3 3 6 3 7 5 3 5 3 3 9 7 1 9 1 9 2 9 3 3 9 3 2 0 1 2 7 6 3 6 1 3 5 6
 4 9 1 5 2 3 1 3 3 3 3 7 1 6 7 1 6 4 8 7 2 5 3 2 1 3 6 4 6 4 6 1 3 2 6 3 2
 6 0 6 4 6 4 2 4 2 6 1 3 6 3 5 2 3 6 2 4 7 9 3 1 7 5 7 6 3 2 4 6 1 3 3 4 3
 4 4 1 7 6 5 7 4 6 9 7 9 3 1 1 6 1 8 6 9 4 0 4 3 3 2 4 4 1 9 2 5 6 1 1 6 6
 7 0 5 7 3 5 9 1 2 7 6 3 1 4 6 4 8 4 6 1 1 5 6 6 7 5 3 3 1 3 2 3 2 0 3 0 7
 4 6 8 5 0 4 1 4 6 3 5 6 7 6 3 6 3 3 7 0 2 4 1 6 6 6 6 6 6 2 0 7 6 1 3 0 2
 0 8 6 6 3 5 1 8 4 2 2 3 9 0 3 0 4 7 5 2 8 1 3 9 2 3 5 6 1 8 3 5 4 1 3 2 8
 6 7 6 3 0 5 4 7 6 1 1 4 2 4 9 2 5 7 1 7 3 2 3 9 3 4 6 3 1 1 1 3 7 1 7 6 7
 4 2 6 7 8 3 4 3 6 7 3 6 7 4 3 4 1 3 1 2 3 2 6 1 3 6 4 3 6 4 3 7 6 5 4 2 2
 7 9 9 4 6 1 2 9 6 4 1 5 3 4 6 6 3 1 1 6 1 3 4 9 0 9 7 2 8 7 9 4 6 3 6 6 4
 3 4 6 6 5 4 5 6 4 3 3 6 3 6 4 6 6 6 4 2 2 2 8 3 1 6 4 2 4 1 1 1 3 7 6 4 3
 7 3 5 7 4 6 8 6 1 7 1 2 7 4 6 6 6 6 3 3 6 7 3 9 3 0 6 7 7 0 7 4 6 6 3 4 7
 6 3 4 8 4 4 5 1 1 3 4 4 7 1 7 5 3 6 6 9 1 6 0 4 7 4 7 2 3 3 6 3 6 1 8 2 3
 9 2 9 4 3 3 1 3 6

In [221]:
result=[]
for i in range(len(Test)):
    result.append([Test[i], preds[i]])

Csv = pd.DataFrame(result)
Csv.to_csv("Result.csv")

In [218]:
preds = ['4' if x==0 else x for x in preds]
preds = ['10' if x==1 else x for x in preds]
preds = ['9' if x==2 else x for x in preds]
preds = ['3' if x==3 else x for x in preds]
preds = ['1' if x==4 else x for x in preds]
preds = ['8' if x==5 else x for x in preds]
preds = ['6' if x==6 else x for x in preds]
preds = ['2' if x==7 else x for x in preds]
preds = ['7' if x==8 else x for x in preds]
preds = ['5' if x==9 else x for x in preds]

In [219]:
preds = [int(x) for x in preds]

In [220]:
print(preds)

[9, 8, 1, 3, 3, 6, 3, 2, 8, 3, 8, 3, 3, 5, 2, 10, 5, 10, 5, 9, 5, 3, 3, 5, 3, 9, 4, 10, 9, 2, 6, 3, 6, 10, 3, 8, 6, 1, 5, 10, 8, 9, 3, 10, 3, 3, 3, 3, 2, 10, 6, 2, 10, 6, 1, 7, 2, 9, 8, 3, 9, 10, 3, 6, 1, 6, 1, 6, 10, 3, 9, 6, 3, 9, 6, 4, 6, 1, 6, 1, 9, 1, 9, 6, 10, 3, 6, 3, 8, 9, 3, 6, 9, 1, 2, 5, 3, 10, 2, 8, 2, 6, 3, 9, 1, 6, 10, 3, 3, 1, 3, 1, 1, 10, 2, 6, 8, 2, 1, 6, 5, 2, 5, 3, 10, 10, 6, 10, 7, 6, 5, 1, 4, 1, 3, 3, 9, 1, 1, 10, 5, 9, 8, 6, 10, 10, 6, 6, 2, 4, 8, 2, 3, 8, 5, 10, 9, 2, 6, 3, 10, 1, 6, 1, 7, 1, 6, 10, 10, 8, 6, 6, 2, 8, 3, 3, 10, 3, 9, 3, 9, 4, 3, 4, 2, 1, 6, 7, 8, 4, 1, 10, 1, 6, 3, 8, 6, 2, 6, 3, 6, 3, 3, 2, 4, 9, 1, 10, 6, 6, 6, 6, 6, 6, 9, 4, 2, 6, 10, 3, 4, 9, 4, 7, 6, 6, 3, 8, 10, 7, 1, 9, 9, 3, 5, 4, 3, 4, 1, 2, 8, 9, 7, 10, 3, 5, 9, 3, 8, 6, 10, 7, 3, 8, 1, 10, 3, 9, 7, 6, 2, 6, 3, 4, 8, 1, 2, 6, 10, 10, 1, 9, 1, 5, 9, 8, 2, 10, 2, 3, 9, 3, 5, 3, 1, 6, 3, 10, 10, 10, 3, 2, 10, 2, 6, 2, 1, 9, 6, 2, 7, 3, 1, 3, 6, 2, 3, 6, 2, 1, 3, 1, 10, 3, 10, 9, 3, 9, 6, 1

In [ ]:
import shutil
shutil.make_archive('/content/drive/MyDrive/indoorTrain', 'zip', '/content/train')

'/content/drive/MyDrive/indoorTrain.zip'